* [#97](https://github.com/salgo60/ProjectOutdoorGyms/issues/97)
* Notebook [97_outdoorgym](https://github.com/salgo60/ProjectOutdoorGyms/tree/main/Jupyter/97_outdoorgym.ipynb)

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-12-12 13:21:20


In [4]:
# current dir 
import os
os.getcwd()

'/Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter'

In [7]:
import os
import requests
import zipfile
import geopandas as gpd

# -------------------------------------------------------
# 1. Ladda ned ZIP-filen från Nacka
# -------------------------------------------------------

url = "https://infobank.nacka.se/Ext/oppnadata/utegym.zip"
zip_path = "/Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter/Nackautegym.zip"


print("Laddar ner utegym.zip ...")
r = requests.get(url)
with open(zip_path, "wb") as f:
    f.write(r.content)

print("Nedladdning klar:", zip_path)

# -------------------------------------------------------
# 2. Packa upp ZIP-filen
# -------------------------------------------------------

extract_dir = "/Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter/utegymNacka"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)

print("Uppackat till:", extract_dir)
print("Innehåll:", os.listdir(extract_dir))

# -------------------------------------------------------
# 3. Identifiera vilka GIS-format som finns
# -------------------------------------------------------

gpkg_file = None
shp_file = None

for root, dirs, files in os.walk(extract_dir):
    for f in files:
        if f.lower().endswith(".gpkg"):
            gpkg_file = os.path.join(root, f)
        if f.lower().endswith(".shp"):
            shp_file = os.path.join(root, f)

print("Hittad GeoPackage:", gpkg_file)
print("Hittad Shapefile:", shp_file)

# -------------------------------------------------------
# 4. Läs in geodata i GeoPandas
# -------------------------------------------------------

if gpkg_file:
    print("Läser GeoPackage...")
    gdf_local = gpd.read_file(gpkg_file)
elif shp_file:
    print("Läser Shapefile...")
    gdf_local = gpd.read_file(shp_file)
else:
    raise FileNotFoundError("Varken GeoPackage eller Shapefile hittades i ZIP-filen.")

# Säkerställ WGS84
gdf_local = gdf_local.to_crs("EPSG:4326")

print("Klar! Antal rader:", len(gdf_local))
gdf_local.head()



Laddar ner utegym.zip ...
Nedladdning klar: /Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter/Nackautegym.zip
Uppackat till: /Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter/utegymNacka
Innehåll: ['Shape', 'Metadata.txt', 'Användarvillkor.txt', 'dxf', 'Friskrivning.txt', 'Geopackage']
Hittad GeoPackage: /Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter/utegymNacka/Geopackage/utegym.gpkg
Hittad Shapefile: /Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter/utegymNacka/Shape/utegym.shp
Läser GeoPackage...
Klar! Antal rader: 18


,namn,nummer,kommundel,ansvar_skotsel,pkid,ansvar,atgardstyp,typ,area,etikett,inlagd_den,andrad_den,lank,Renoverad,Longitud,Latitud,Adresspunkt,geometry
0,Nyckelviken,NaN,None,None,76489,None,None,utegym,475.908099,GYM,2022-11-01,2021-05-31,https://www.nacka.se/uppleva--gora/friluftsliv...,NaT,NaN,NaN,Nyckelviksvägen,POINT (18.17995 59.31783)
1,Velamsund,NaN,None,None,76491,None,None,utegym,625.345713,GYM,2018-11-01,2022-11-01,https://www.nacka.se/uppleva--gora/friluftsliv...,NaT,NaN,NaN,Velamsundsvägen 33,POINT (18.32001 59.34672)
2,Hellasgården,NaN,None,None,76602,Stockholm,None,utegym,725.504116,GYM,2018-11-06,2022-10-26,https://www.nacka.se/uppleva--gora/friluftsliv...,NaT,NaN,NaN,None,POINT (18.15909 59.28984)
3,Sickla strand,NaN,None,None,76448,None,None,utegym,52.616543,GYM,2018-11-01,2022-11-01,https://www.nacka.se/uppleva--gora/friluftsliv...,NaT,NaN,NaN,Sickla strand 25,POINT (18.1216 59.30231)
4,Hedvigslund,NaN,None,None,76422,None,None,utegym,77.625551,GYM,2018-11-01,2022-11-01,https://www.nacka.se/uppleva--gora/friluftsliv...,NaT,NaN,NaN,Anemonvägen 38,POINT (18.18702 59.24968)


In [18]:
import folium
from folium.features import DivIcon
# Beräkna medelpunkt
mean_lat = gdf_local.geometry.y.mean()
mean_lon = gdf_local.geometry.x.mean()

m = folium.Map(location=[mean_lat, mean_lon], zoom_start=12, tiles="OpenStreetMap",
    width=300,
    height=300)
# Lägg in varje utegym som marker
for idx, row in gdf_local.iterrows():
    lat = row.geometry.y
    lon = row.geometry.x

    name = None
    for col in ["name", "namn", "Namn", "NAME"]:
        if col in gdf_local.columns:
            name = row[col]
            break

    popup_text = f"{name}" if name else f"Utegym {idx}"

    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color="green", icon="dumbbell", prefix="fa")
    ).add_to(m)
# Rendera Folium-kartan till HTML (ren HTML-snutt, ingen separat fil)
map_html = m.get_root().render()


In [19]:
import pandas as pd
from datetime import datetime

# Kontroll att gdf_local finns
gdf_local.head()

# ---------------------------------------------------------
# 1. Skapa datum för filnamn och rapport
# ---------------------------------------------------------

date_tag = datetime.now().strftime("%Y_%m_%d")
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

output_path = f"./utegym_report_Nacka_{date_tag}.html"

# ---------------------------------------------------------
# 2. Metadata
# ---------------------------------------------------------

row_count = len(gdf_local)
crs = gdf_local.crs.to_string() if gdf_local.crs else "None"
columns = ", ".join(gdf_local.columns)

preview_html = gdf_local.head(20).to_html()

# ---------------------------------------------------------
# 3. Bygg HTML-rapport
# ---------------------------------------------------------

html_report = f"""
<html>
<head>
    <meta charset="UTF-8">
    <title>Utegymrapport {date_tag}</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 40px;
        }}
        h1, h2 {{
            color: #333;
        }}
        a {{
            color: #1a0dab;
        }}
        .section {{
            margin-bottom: 30px;
        }}
        table {{
            border-collapse: collapse;
            width: 100%;
        }}
        table, th, td {{
            border: 1px solid #ccc;
        }}
        th, td {{
            padding: 8px;
        }}
        th {{
            background-color: #f0f0f0;
        }}
    </style>
</head>
<body>

<h1>Utegymrapport</h1>
<p><strong>Issue:</strong> <a href="https://github.com/salgo60/ProjectOutdoorGyms/issues/97">#97</a></p>
<p>Genererad: <strong>{timestamp}</strong></p>
<div class="section">
    <h2>Karta över utegymmen</h2>
    {map_html}
</div>

<div class="section">
    <h2>Datasetinformation</h2>
    <p><strong>Antal rader:</strong> {row_count}</p>
    <p><strong>CRS:</strong> {crs}</p>
    <p><strong>Kolumner:</strong> {columns}</p>
</div>

<div class="section">
    <h2>Förhandsgranskning (20 första raderna)</h2>
    {preview_html}
</div>

</body>
</html>
"""

# ---------------------------------------------------------
# 4. Spara HTML-filen
# ---------------------------------------------------------

with open(output_path, "w", encoding="utf-8") as f:
    f.write(html_report)

output_path


'./utegym_report_Nacka_2025_12_12.html'

In [2]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-12-12 13:21:32
Total time elapsed: 00 minutes 12.20 seconds
